# C2. Barrier Option in Libor Market Model 
# Barrier cap/floor

## Barrier Caplet Pricing Algorithms
Compare:
1. Analytic closed‑form (\~V)  
2. Algorithm 2.1 (weak 1, kick‑back)  
3. Algorithm 2.2 (weak ½, no VR)  
=> Sensitivity analysis
4. Algorithm 2.2 with variance reduction

$$
V_{\mathrm{caplet}}(t)
= \delta \, P\bigl(t,\,T_{i+1}\bigr)
\;\mathbb{E}^{\mathbb{Q}^{T_{i+1}}}
\Bigl[\bigl(L^i(T_i)-K\bigr)_+\;\chi\{\theta>T_i\}\,\bigm|\;\mathcal{F}_t\Bigr]
$$

#### Imports

In [1]:
!pip install matplotlib pandas plotly seaborn

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
import plotly.graph_objects as go
from tqdm import tqdm
from scipy.stats import norm
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.colors as mcolors
import copy

In [3]:
L0, H, K = 0.13, 0.28, 0.01
sigma, T = 0.25, 9.0
N_paths = 300000
h_list  = [0.25, 0.125, 0.0625, 0.03125, 0.015625] #[0.25, 0.125, 0.0625]

## Analytic closed‑form 

In [4]:
def analytic_vtilde_caplet(L, H, K, sigma, tau):
    v = sigma * np.sqrt(tau)
    def d_plus(x):  return (np.log(x) + 0.5*v**2)/v
    def d_minus(x): return (np.log(x) - 0.5*v**2)/v

    term1 = L*(norm.cdf(d_plus(L/K)) - norm.cdf(d_plus(L/H)))
    term2 = -K*(norm.cdf(d_minus(L/K)) - norm.cdf(d_minus(L/H)))
    term3 = -H*(norm.cdf(d_plus(H*H/(K*L))) - norm.cdf(d_plus(H/L)))
    term4 = K*L/H*(norm.cdf(d_minus(H*H/(K*L))) - norm.cdf(d_minus(H/L)))
    return term1 + term2 + term3 + term4

In [5]:
exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact

np.float64(0.06571345192162645)

# Algorithm 2.1 – weak order 1 with kick‑back reflection

In [6]:
def simulate_algo1(L0, H, K, sigma, T, h, N_paths, seed=42):
    """
    Algorithm 2.1 (weak 1, kick‐back reflection).
    Returns (price, stderr, mean_exit_time).
    """
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T / h))
    lnH = np.log(H)
    sqrt_h = np.sqrt(h)
    half_sigma2_h = 0.5 * sigma**2 * h
    # λ√h from (4.12)
    lambda_sqrt_h = -half_sigma2_h + sigma*sqrt_h

    payoffs = np.zeros(N_paths)
    exit_steps = np.zeros(N_paths, dtype=int)

    for i in tqdm(range(N_paths), desc="MC paths"):
        lnL = np.log(L0)
        for k in range(M):
            if lnL >= lnH + half_sigma2_h - sigma*sqrt_h:
                # in boundary zone
                delta = lnH - lnL
                p = lambda_sqrt_h / (delta + lambda_sqrt_h)
                if rng.random() < p:
                    # knock‐out at barrier
                    exit_steps[i] = k
                    lnL = lnH
                    break
                else:
                    # kick back in
                    lnL -= lambda_sqrt_h
                    xi = 1 if rng.random() < 0.5 else -1
                    lnL += -half_sigma2_h + sigma*sqrt_h*xi
            else:
                # usual Euler step
                xi = 1 if rng.random() < 0.5 else -1
                lnL += -half_sigma2_h + sigma*sqrt_h*xi
        else:
            # survived to maturity
            exit_steps[i] = M
            payoffs[i] = max(np.exp(lnL) - K, 0.0)

    price = payoffs.mean()
    stderr = payoffs.std(ddof=1) / np.sqrt(N_paths)
    mean_exit_time = exit_steps.mean() * h
    return price, stderr, mean_exit_time

In [7]:
results_algo1 = []

for h in h_list:
    p, se, _   = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

results_algo1

MC paths:  19%|█▉        | 56261/300000 [00:04<00:18, 13248.15it/s]


KeyboardInterrupt: 

## Algorithm 2.2 – weak order 1/2, absorbing barrier (no VR)

In [ ]:
def simulate_algo2(L0, H, K, sigma, T, h, N_paths, seed=42):
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T/h))
    lnH, sqrt_h = np.log(H), np.sqrt(h)
    half_sig2h = 0.5 * sigma**2 * h
    payoffs = np.zeros(N_paths)
    for i in tqdm(range(N_paths), desc="MC paths"):
        lnL = np.log(L0)
        for k in range(M):
            if lnL >= lnH + half_sig2h - sigma*sqrt_h:
                break
            xi = 1 if rng.random() < 0.5 else -1
            lnL += -half_sig2h + sigma*sqrt_h*xi
        payoffs[i] = max(np.exp(lnL)-K, 0.0) if k == M-1 else 0.0
    price, stderr = payoffs.mean(), payoffs.std(ddof=1)/np.sqrt(N_paths)
    return price, stderr

In [ ]:
results_algo2 = []

for h in h_list:
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

results_algo2

MC paths: 100%|██████████| 300000/300000 [02:47<00:00, 1792.85it/s]


[{'h': 0.25,
  'Price': np.float64(0.06257531070865202),
  'CI': np.float64(0.00020400285018166406)},
 {'h': 0.125,
  'Price': np.float64(0.06299697305907437),
  'CI': np.float64(0.00020131107854003962)},
 {'h': 0.0625,
  'Price': np.float64(0.06321458719352833),
  'CI': np.float64(0.00019953209213402877)},
 {'h': 0.03125,
  'Price': np.float64(0.06410563900060298),
  'CI': np.float64(0.00020093403136147898)},
 {'h': 0.015625,
  'Price': np.float64(0.06450418164996352),
  'CI': np.float64(0.0002018366595446813)}]

In [ ]:
# Prepare figure
fig = go.Figure()

# Add trace for Algorithm 1
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    name='Algorithm 1 - weak order 1 <br> with kick-back reflection'
))

# Add trace for Algorithm 2
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    name='Algorithm 2 - weak order 1/2 <br> with absorbing barrier'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo2),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact'
))

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price from Algorithm 1 vs Algorithm 2 with varying step size<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"L0={L0}, H={H}, K={K}, σ={sigma}, T={T}, "
            f"N_paths={N_paths}"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

From the plot above, it’s evident that Algorithm 1 (kick-back reflection) vastly outperforms Algorithm 2 (absorbing barrier). Even at the coarsest time step (h=0.25), Algorithm 1’s estimates lie almost exactly on the true caplet price (dotted line) and its confidence intervals remain uniformly small, reflecting its higher weak order and stability. By contrast, Algorithm 2 is heavily biased downward—starting near 0.0645—and only inches toward the exact value as h decreases. 

The familiar oscillatory pattern of the binary-tree scheme is visible in Algorithm 1’s curve but fades as the mesh is refined, whereas Algorithm 2’s convergence is monotonic yet slow. 

**In short, Algorithm 1 delivers superior accuracy and greater efficiency; Algorithm 2 requires prohibitively small steps to even approach the correct price.**

## Sensitivity analysis
Impact of changing L0, H, K, σ, T - one parameter at a time

In [ ]:
## Normal color
# get the default Seaborn palette (first color is blue)
blue_rgb = sns.color_palette()[0]

# convert (r,g,b) floats → hex string
blue_hex = mcolors.to_hex(blue_rgb)

orange_rgb = sns.color_palette()[1]
orange_hex = mcolors.to_hex(orange_rgb)

def lighten_color(hex_color, amount=0.5):
    """
    Lighten a given color by mixing it with white.
    amount=0   → original color
    amount=1   → white
    """
    # convert hex to (r, g, b)
    r, g, b = mcolors.to_rgb(hex_color)
    # blend each channel toward 1 (white)
    r_l = r + (1 - r) * amount
    g_l = g + (1 - g) * amount
    b_l = b + (1 - b) * amount
    return mcolors.to_hex((r_l, g_l, b_l))

## Lighten
blue_pastel_hex = lighten_color(blue_hex, amount=0.4)
orange_pastel_hex = lighten_color(orange_hex, amount=0.4)


In [ ]:
N_paths = 30_000
h_list = [0.25, 0.0625, 0.015625]

# perturbed values (one at a time)
L0, H, K = 0.13, 0.28, 0.01
sigma, T = 0.25, 9.0
L0_, H_, K_ = 0.08, 0.20, 0.005
sigma_, T_ = 0.30, 7.0

In [ ]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0_, H, K, sigma, T, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0_, H, K, sigma, T, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0_, H, K, sigma, T)

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 51163.36it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 15483.52it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:07<00:00, 3825.11it/s]


In [ ]:
# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to L0<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"L0={L0} vs L0_={L0_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

In [ ]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0, H_, K, sigma, T, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H_, K, sigma, T, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0, H_, K, sigma, T)

# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to H<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"H={H} vs H_={H_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 80034.35it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 21500.07it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:05<00:00, 5494.15it/s]


In [ ]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0, H, K_, sigma, T, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K_, sigma, T, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0, H, K_, sigma, T)

# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to K<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"K={K} vs K_={K_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 64424.97it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 17130.10it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:07<00:00, 4106.21it/s]


In [ ]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0, H, K, sigma_, T, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma_, T, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0, H, K, sigma_, T)

# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to sigma<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"sigma={sigma} vs sigma_={sigma_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 67339.79it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 18310.56it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:06<00:00, 4370.72it/s]


In [ ]:
results_algo2 = []
results_algo2_ = []
results_algo1 = []
results_algo1_ = []

for h in h_list:
    print(h, "\n")
    p, se = simulate_algo2(L0, H, K, sigma, T, h, N_paths)
    results_algo2.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se = simulate_algo2(L0, H, K, sigma, T_, h, N_paths)
    results_algo2_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T, h, N_paths)
    results_algo1.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })
    p, se, _ = simulate_algo1(L0, H, K, sigma, T_, h, N_paths)
    results_algo1_.append({
        'h': h,
        'Price': p,
        'CI': 1.96*se
    })

exact = analytic_vtilde_caplet(L0, H, K, sigma, T)
exact_ = analytic_vtilde_caplet(L0, H, K, sigma, T_)

# Prepare figure
fig = go.Figure()

###### Baselines
fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact]*len(results_algo1),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1],
    y=[d['Price'] for d in results_algo1],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1], visible=True),
    line=dict(color=blue_hex),
    marker=dict(color=blue_hex),
    name='Algorithm 1 - baseline'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2],
    y=[d['Price'] for d in results_algo2],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2], visible=True),
    line=dict(color=orange_hex),
    marker=dict(color=orange_hex),
    name='Algorithm 2 - baseline'
))

###### Perturbed

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],   # or use a combined h-list like [0.25,0.125,0.0625]
    y=[exact_]*len(results_algo1_),
    mode='lines',
    line=dict(dash='dot', color='black'),
    name='Exact perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo1_],
    y=[d['Price'] for d in results_algo1_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo1_], visible=True),
    line=dict(color=blue_pastel_hex),
    marker=dict(color=blue_pastel_hex),
    name='Algorithm 1 - perturbed'
))

fig.add_trace(go.Scatter(
    x=[d['h'] for d in results_algo2_],
    y=[d['Price'] for d in results_algo2_],
    mode='lines+markers',
    error_y=dict(type='data', array=[d['CI'] for d in results_algo2_], visible=True),
    line=dict(color=orange_pastel_hex),
    marker=dict(color=orange_pastel_hex),
    name='Algorithm 2 - perturbed'
))


###### General layout

fig.update_layout(
    template="simple_white",
    title={
        "text": (
            # main title in default size:
            "Caplet price sensitivity to T<br>"
            # parameters in a smaller span (e.g. 12px):
            f"<span style='font-size:12px;'>"
            f"T={T} vs T_={T_} <br> (N_paths = {N_paths})"
            "</span>"
        ),
        "x": 0.5,
        "xanchor": "center"
    }
)

fig.show()

0.25 



MC paths: 100%|██████████| 30000/30000 [00:00<00:00, 75849.89it/s]


0.0625 



MC paths: 100%|██████████| 30000/30000 [00:01<00:00, 20212.13it/s]


0.015625 



MC paths: 100%|██████████| 30000/30000 [00:06<00:00, 4847.48it/s]


## Algorithm 2.2 with variance reduction (add Z from (4.6))

### Variance‑reduction term F(s,L) ≈ -σ ∂V/∂L from (4.7)

In [8]:
def F_vr_fd(L, H, K, sigma, tau, eps):
    """Finite-difference control-variate integrand."""
    Vp = analytic_vtilde_caplet(L + eps, H, K, sigma, tau)
    Vm = analytic_vtilde_caplet(L - eps, H, K, sigma, tau)
    dVdL = (Vp - Vm) / (2 * eps)
    return -sigma * dVdL

In [9]:
def F_vr_exact(L, H, K, sigma, tau, eps=None):
    """Exact closed-form ∂_L Ṽ from the barrier-caplet formula."""
    v = sigma * np.sqrt(tau)
    inv_v = 1.0 / v
    lnL = np.log(L)

    # deltas
    a1 = (lnL - np.log(K) + 0.5*v*v) * inv_v
    a2 = (lnL - np.log(H) + 0.5*v*v) * inv_v
    b1 = (lnL - np.log(K) - 0.5*v*v) * inv_v
    b2 = (lnL - np.log(H) - 0.5*v*v) * inv_v
    c1 = (np.log(H*H/(K*L)) + 0.5*v*v) * inv_v
    c2 = (np.log(H/L)    + 0.5*v*v) * inv_v
    d1 = (np.log(H*H/(K*L)) - 0.5*v*v) * inv_v
    d2 = (np.log(H/L)    - 0.5*v*v) * inv_v

    phi = norm.pdf
    Phi = norm.cdf

    T1 = (Phi(a1) - Phi(a2)) + inv_v*(phi(a1) - phi(a2))
    T2 = -(K/(L*v))*(phi(b1) - phi(b2))
    T3 =  (H/(L*v))*(phi(c1) - phi(c2))
    T4 = (K/H)*(Phi(d1) - Phi(d2)) - (K/(H*v))*(phi(d1) - phi(d2))

    dVdL = T1 + T2 + T3 + T4
    return -sigma * dVdL

In [10]:
def simulate_algo2(
    L0, H, K, sigma, T, h, N_paths,
    vr_method: str = "none",   # "none", "fd" or "exact"
    seed: int = 42,
    eps: float = 1e-4
):
    """
    Monte-Carlo Algo2 with choice of variance-reduction:

      vr_method="none"  → no control-variate
      vr_method="fd"    → finite-difference CV (needs eps)
      vr_method="exact" → exact CV (closed-form)
    """
    rng = np.random.default_rng(seed)
    M = int(np.ceil(T / h))
    lnH, sqrt_h = np.log(H), np.sqrt(h)
    half_sig2h = 0.5 * sigma**2 * h

    payoffs = np.zeros(N_paths)
    if vr_method != "none":
        Z_vals = np.zeros(N_paths)

    for i in tqdm(range(N_paths), desc="MC paths"):
        lnL = np.log(L0)
        Z = 0.0

        for k in range(M):
            tau = T - k*h
            # boundary check
            if lnL >= lnH + half_sig2h - sigma*sqrt_h:
                break

            xi = 1 if rng.random() < 0.5 else -1
            lnL += -half_sig2h + sigma*sqrt_h*xi

            if vr_method == "fd":
                Fvr = F_vr_fd(np.exp(lnL), H, K, sigma, tau, eps)
                Z += Fvr * sqrt_h * xi
            elif vr_method == "exact":
                Fvr = F_vr_exact(np.exp(lnL), H, K, sigma, tau)
                Z += Fvr * sqrt_h * xi

        payoff = max(np.exp(lnL) - K, 0.0) if k == M-1 else 0.0
        payoffs[i] = payoff
        if vr_method != "none":
            Z_vals[i] = Z

    # No control-variate: plain MC
    if vr_method == "none":
        price  = payoffs.mean()
        stderr = payoffs.std(ddof=1) / np.sqrt(N_paths)
        return price, stderr

    # Control-variate adjustment
    cov = np.cov(payoffs, Z_vals, ddof=1)[0,1]
    varZ = Z_vals.var(ddof=1)
    g    = cov / varZ
    Z_bar = Z_vals.mean()

    adj = payoffs - g*(Z_vals - Z_bar)
    price  = adj.mean()
    stderr = adj.std(ddof=1) / np.sqrt(N_paths)
    return price, stderr, g

In [11]:
h = 0.25

In [12]:
# 1) Plain MC
p0, e0 = simulate_algo2(L0, H, K, sigma, T, h, 3000, vr_method="none")

MC paths: 100%|██████████| 3000/3000 [00:00<00:00, 19012.73it/s]


In [13]:
# 2) FD-based CV
p_fd, e_fd, g_fd = simulate_algo2(
    L0, H, K, sigma, T, h, 3000,
    vr_method="fd", eps=1e-4
)

MC paths: 100%|██████████| 3000/3000 [01:14<00:00, 40.20it/s]


In [14]:
# 3) Exact-derivative CV
p_ex, e_ex, g_ex = simulate_algo2(
    L0, H, K, sigma, T, h, 3000,
    vr_method="exact"
)

MC paths: 100%|██████████| 3000/3000 [00:55<00:00, 54.14it/s]


In [15]:
print(f"plain: {p0:.6f} ± {e0:.6f}")
print(f"FD-CV:  {p_fd:.6f} ± {e_fd:.6f} (g={g_fd:.4f})")
print(f"Exact:  {p_ex:.6f} ± {e_ex:.6f} (g={g_ex:.4f})")

plain: 0.062768 ± 0.001029
FD-CV:  0.062768 ± 0.000965 (g=-0.0479)
Exact:  0.062768 ± 0.000965 (g=-0.0479)


## Algorithm 2.2 with variance reduction (add Z from (4.6)) + NUMBA Acceleration

In [18]:
!pip install numba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 9.3 MB/s eta 0:00:0000:0100:01


In [ ]:
import numpy as np
import numba
from numba import njit, prange
import math
from tqdm import tqdm # Keep tqdm for the outer loop if desired, but remove from Numba part

# Parameters from the notebook (adjust N_paths as needed)
L0, H, K = 0.13, 0.28, 0.01
sigma, T = 0.25, 9.0
# N_paths = 3000 # Original small test
# N_paths = 300000 # Original large run
N_paths = 1_000_000 # Target for 1 hour run
h = 0.015625 # Using one of the smaller step sizes for demonstration
seed = 42
eps = 1e-4 # For FD method if used (though exact is preferred)

# ==================================================
# 1. Numba-compatible helper functions
# ==================================================

SQRT_2 = np.sqrt(2.0)
SQRT_2PI = np.sqrt(2.0 * np.pi)

@njit(fastmath=True)
def norm_pdf_numba(x):
    """Numba-compatible standard normal PDF."""
    return math.exp(-0.5 * x * x) / SQRT_2PI

@njit(fastmath=True)
def norm_cdf_numba(x):
    """Numba-compatible standard normal CDF using math.erf."""
    return 0.5 * (1.0 + math.erf(x / SQRT_2))




Parameters: L0=0.13, H=0.28, K=0.01, sigma=0.25, T=9.0, h=0.015625, N_paths=1000000
Starting Numba simulation for 1000000 paths...
Numba simulation finished.
Calculating final price/stderr (no VR)...

Numba Plain MC Result:
Price: 0.06437745 ± 0.00005615
Time:  2.91 seconds
Simulations per second: 344203
------------------------------
Starting Numba simulation for 1000000 paths...
Numba simulation finished.
Calculating final price/stderr with Control Variate...
CV calculation finished.

Numba Exact CV Result:
Price: 0.06438229 ± 0.00002987 (g=-0.3936)
Time:  22.08 seconds
Simulations per second: 45284
------------------------------

Variance Reduction Factor (Exact CV vs Plain): 3.53x

Achieved simulations per hour (Exact CV): 163,023,515
Target simulations per hour:             1,000,000
>>> Target performance MET or EXCEEDED! <<<

Estimated speedup vs original (50 sims/sec): 905.7x
>>> Target speedup (5x) ACHIEVED! <<<


In [ ]:
# ==================================================
# 2. Numba-compatible core logic functions
# ==================================================

@njit(fastmath=True)
def analytic_vtilde_caplet_numba(L, H, K, sigma, tau):
    """Numba-compatible version of analytic_vtilde_caplet."""
    if L <= 0 or K <= 0 or H <= 0 or tau <= 0 or L >= H: # Add checks for invalid inputs
        return 0.0
    v = sigma * math.sqrt(tau)
    if v == 0: # Avoid division by zero
        return max(L - K, 0.0) if L < H else 0.0

    log_L = math.log(L)
    log_H = math.log(H)
    log_K = math.log(K)
    half_v2 = 0.5 * v * v

    # Calculate d+ and d- terms carefully to avoid log(0) or division by zero
    d_plus_LK = (log_L - log_K + half_v2) / v
    d_plus_LH = (log_L - log_H + half_v2) / v
    d_minus_LK = (log_L - log_K - half_v2) / v
    d_minus_LH = (log_L - log_H - half_v2) / v

    log_HHKL = 2.0 * log_H - log_K - log_L
    log_HL = log_H - log_L

    d_plus_HHKL = (log_HHKL + half_v2) / v
    d_plus_HL = (log_HL + half_v2) / v
    d_minus_HHKL = (log_HHKL - half_v2) / v
    d_minus_HL = (log_HL - half_v2) / v


    term1 = L * (norm_cdf_numba(d_plus_LK) - norm_cdf_numba(d_plus_LH))
    term2 = -K * (norm_cdf_numba(d_minus_LK) - norm_cdf_numba(d_minus_LH))
    term3 = -H * (norm_cdf_numba(d_plus_HHKL) - norm_cdf_numba(d_plus_HL))
    term4 = K * L / H * (norm_cdf_numba(d_minus_HHKL) - norm_cdf_numba(d_minus_HL))

    # Ensure result is non-negative (can happen due to floating point issues near boundaries)
    result = term1 + term2 + term3 + term4
    return max(result, 0.0)


@njit(fastmath=True)
def F_vr_exact_numba(L, H, K, sigma, tau):
    """Numba-compatible exact closed-form ∂_L Ṽ."""
    if L <= 0 or K <= 0 or H <= 0 or tau <= 0 or L >= H: # Add checks for invalid inputs
        return 0.0
    v = sigma * math.sqrt(tau)
    if v == 0: # Avoid division by zero
        return 0.0 # Derivative is 0 if v=0 (or handle based on specific limits)

    inv_v = 1.0 / v
    log_L = math.log(L)
    log_H = math.log(H)
    log_K = math.log(K)
    half_v2 = 0.5 * v * v

    # Calculate terms for dVdL
    a1 = (log_L - log_K + half_v2) * inv_v
    a2 = (log_L - log_H + half_v2) * inv_v
    b1 = (log_L - log_K - half_v2) * inv_v
    b2 = (log_L - log_H - half_v2) * inv_v
    c1 = (2.0 * log_H - log_K - log_L + half_v2) * inv_v
    c2 = (log_H - log_L + half_v2) * inv_v
    d1 = (2.0 * log_H - log_K - log_L - half_v2) * inv_v
    d2 = (log_H - log_L - half_v2) * inv_v

    # Use Numba-compatible pdf/cdf
    phi1 = norm_pdf_numba(a1)
    phi2 = norm_pdf_numba(a2)
    Phi1 = norm_cdf_numba(a1)
    Phi2 = norm_cdf_numba(a2)

    phib1 = norm_pdf_numba(b1)
    phib2 = norm_pdf_numba(b2)

    phic1 = norm_pdf_numba(c1)
    phic2 = norm_pdf_numba(c2)

    phid1 = norm_pdf_numba(d1)
    phid2 = norm_pdf_numba(d2)
    Phid1 = norm_cdf_numba(d1)
    Phid2 = norm_cdf_numba(d2)

    # Calculate terms of dVdL
    T1 = (Phi1 - Phi2) + inv_v * (phi1 - phi2)
    T2 = -(K / (L * v)) * (phib1 - phib2)
    T3 = (H / (L * v)) * (phic1 - phic2)
    T4 = (K / H) * (Phid1 - Phid2) - (K / (H * v)) * (phid1 - phid2)

    dVdL = T1 + T2 + T3 + T4
    return -sigma * dVdL

In [ ]:
# ==================================================
# 3. Numba-Optimized Simulation Function
# ==================================================

@njit(parallel=True, fastmath=True)
def simulate_algo2_numba_internal(
    L0, H, K, sigma, T, h, N_paths, vr_method_flag # 0:none, 1:exact
):
    """
    Internal Numba-compiled simulation loop.
    Uses parallel=True for potential speedup across paths.
    """
    M = int(np.ceil(T / h))
    lnL0 = math.log(L0)
    lnH = math.log(H)
    sqrt_h = math.sqrt(h)
    half_sig2h = 0.5 * sigma * sigma * h
    barrier_check_level = lnH + half_sig2h - sigma * sqrt_h

    # Output arrays initialized with zeros
    payoffs = np.zeros(N_paths)
    Z_vals = np.zeros(N_paths) # Always calculate Z space, use later if vr_method != 0

    # Parallel loop over paths
    for i in prange(N_paths):
        lnL = lnL0
        Z = 0.0
        path_active = True

        for k in range(M):
            # Check barrier condition FIRST
            if lnL >= barrier_check_level:
                path_active = False
                break # Exit inner loop for this path

            # Generate random step for this path and step
            # Use Numba's recommended way for parallel RNG: get state per thread
            # Simpler for now: use standard np.random inside loop (less ideal for perfect parallel scaling/reproducibility)
            # A better approach involves passing pre-generated random numbers array.
            # Using simple approach for now:
            xi = np.random.randint(0, 2) * 2 - 1 # Generates -1 or 1

            # Calculate next step lnL
            lnL_next = lnL - half_sig2h + sigma * sqrt_h * xi

            # Calculate Control Variate Increment (if applicable) BEFORE updating lnL
            if vr_method_flag == 1: # 1 means "exact"
                 # Calculate tau remaining for the *current* step k
                 # The CV term F depends on the state *before* the random jump,
                 # but the paper's formula Z = sum F(L_k+1) * dW_k uses state *after* jump.
                 # Let's follow the paper's formula (4.6) which uses L(t_k+1)
                 # tau = T - (k + 1) * h # Time remaining *after* the step
                 # if tau > 1e-10: # Ensure tau is positive
                 #     L_next = math.exp(lnL_next)
                 #     # Check if L_next is valid before calling F_vr
                 #     if L_next < H and L_next > 0:
                 #         Fvr = F_vr_exact_numba(L_next, H, K, sigma, tau)
                 #         Z += Fvr * sqrt_h * xi

                 # *** Correction based on typical CV application & formula (4.6) ***
                 # The control variate Z = sum_{k=0}^{M-1} F(t_k, L(t_k)) * (L(t_{k+1}) - L(t_k) - drift)
                 # Or simpler Z = sum F * dW = sum F * sigma * sqrt(h) * xi
                 # The function F should be evaluated at the *start* of the interval (t_k, L(t_k))
                 tau = T - k * h # Time remaining at the start of the step
                 if tau > 1e-10:
                     L_current = math.exp(lnL)
                     if L_current < H and L_current > 0: # Ensure valid state
                         Fvr = F_vr_exact_numba(L_current, H, K, sigma, tau)
                         Z += Fvr * sigma * sqrt_h * xi # Match dZ = F * dW


            # Update Log Libor rate
            lnL = lnL_next

        # End of inner loop (time steps)

        # Calculate payoff only if path survived until maturity (k == M-1 means loop finished)
        if path_active:
            payoffs[i] = max(math.exp(lnL) - K, 0.0)
        # else: payoff remains 0 (hit barrier)

        if vr_method_flag != 0:
             Z_vals[i] = Z # Store accumulated Z for this path

    return payoffs, Z_vals


def simulate_algo2_numba(
    L0, H, K, sigma, T, h, N_paths,
    vr_method: str = "none",   # "none" or "exact"
    seed: int = 42
):
    """
    Wrapper function for the Numba simulation.
    Handles seeding, calling the Numba kernel, and final CV calculation.
    """
    # Seed NumPy's global random state for Numba (if reproducibility is needed)
    # Note: Seeding global state isn't ideal for perfect parallel reproducibility.
    # For strict reproducibility, generate random numbers outside and pass them in.
    np.random.seed(seed)

    vr_flag = 0
    if vr_method == "exact":
        vr_flag = 1
    elif vr_method != "none":
        raise ValueError("vr_method must be 'none' or 'exact' for Numba version")

    # --- Run the Numba simulation ---
    print(f"Starting Numba simulation for {N_paths} paths...")
    payoffs, Z_vals = simulate_algo2_numba_internal(
        L0, H, K, sigma, T, h, N_paths, vr_flag
    )
    print("Numba simulation finished.")
    # --------------------------------

    # --- Post-processing ---
    # No control-variate: plain MC
    if vr_flag == 0:
        price = np.mean(payoffs)
        # Calculate stderr using ddof=1 for sample standard deviation
        stderr = np.std(payoffs, ddof=1) / np.sqrt(N_paths)
        print("Calculating final price/stderr (no VR)...")
        return price, stderr

    # Control-variate adjustment
    else:
        print("Calculating final price/stderr with Control Variate...")
        # Use ddof=1 for unbiased sample covariance and variance
        # np.cov returns a 2x2 matrix
        cov_matrix = np.cov(payoffs, Z_vals, ddof=1)

        # Handle cases where variance might be zero or near-zero
        varZ = cov_matrix[1, 1]
        if varZ < 1e-20: # Check if variance is effectively zero
             print("Warning: Variance of Z is near zero. Skipping CV adjustment.")
             g = 0.0
             price = np.mean(payoffs) # Fallback to plain MC price
        else:
            cov = cov_matrix[0, 1]
            g = cov / varZ # Optimal control variate coefficient

            # Apply adjustment: V_adj = V - g * (Z - E[Z])
            # E[Z] should be 0 for this martingale control variate, but use sample mean for robustness
            Z_bar = np.mean(Z_vals)
            adj_payoffs = payoffs - g * (Z_vals - Z_bar) # Adjusted payoffs
            price = np.mean(adj_payoffs)


        # Calculate stderr of the *adjusted* payoffs
        # Need to recalculate variance of adjusted payoffs if g was non-zero
        if g != 0.0:
             stderr = np.std(adj_payoffs, ddof=1) / np.sqrt(N_paths)
        else: # If g is zero, stderr is based on original payoffs
             stderr = np.std(payoffs, ddof=1) / np.sqrt(N_paths)


        print("CV calculation finished.")
        return price, stderr, g


In [22]:
# ==================================================
# 4. Running the Simulation
# ==================================================
import time

print(f"Parameters: L0={L0}, H={H}, K={K}, sigma={sigma}, T={T}, h={h}, N_paths={N_paths}")

# --- Run Plain MC (Numba) ---
start_time = time.time()
p0_numba, e0_numba = simulate_algo2_numba(
    L0, H, K, sigma, T, h, N_paths, vr_method="none", seed=seed
)
end_time = time.time()
time_plain = end_time - start_time
print(f"\nNumba Plain MC Result:")
print(f"Price: {p0_numba:.8f} ± {e0_numba:.8f}")
print(f"Time:  {time_plain:.2f} seconds")
print(f"Simulations per second: {N_paths / time_plain:.0f}")
print("-" * 30)


# --- Run Exact CV (Numba) ---
start_time = time.time()
p_ex_numba, e_ex_numba, g_ex_numba = simulate_algo2_numba(
    L0, H, K, sigma, T, h, N_paths, vr_method="exact", seed=seed
)
end_time = time.time()
time_exact = end_time - start_time
print(f"\nNumba Exact CV Result:")
print(f"Price: {p_ex_numba:.8f} ± {e_ex_numba:.8f} (g={g_ex_numba:.4f})")
print(f"Time:  {time_exact:.2f} seconds")
print(f"Simulations per second: {N_paths / time_exact:.0f}")
print("-" * 30)

# --- Variance Reduction Analysis ---
if 'e0_numba' in locals() and 'e_ex_numba' in locals() and e_ex_numba > 0:
    variance_reduction_factor = (e0_numba / e_ex_numba)**2
    print(f"\nVariance Reduction Factor (Exact CV vs Plain): {variance_reduction_factor:.2f}x")
else:
    print("\nCould not calculate variance reduction factor (check standard errors).")

# --- Performance vs Target ---
sims_per_hour_exact = (N_paths / time_exact) * 3600
target_sims_per_hour = 1_000_000
print(f"\nAchieved simulations per hour (Exact CV): {sims_per_hour_exact:,.0f}")
print(f"Target simulations per hour:             {target_sims_per_hour:,.0f}")
if sims_per_hour_exact >= target_sims_per_hour:
    print(">>> Target performance MET or EXCEEDED! <<<")
else:
    print(">>> Target performance NOT met. <<<")

# Compare speedup vs original estimate (3000/min = 50/sec)
original_sims_per_sec = 50
speedup_factor = (N_paths / time_exact) / original_sims_per_sec
print(f"\nEstimated speedup vs original ({original_sims_per_sec:.0f} sims/sec): {speedup_factor:.1f}x")
target_speedup = 5
if speedup_factor >= target_speedup:
     print(f">>> Target speedup ({target_speedup}x) ACHIEVED! <<<")
else:
     print(f">>> Target speedup ({target_speedup}x) NOT achieved. <<<")

Parameters: L0=0.13, H=0.28, K=0.01, sigma=0.25, T=9.0, h=0.015625, N_paths=1000000
Starting Numba simulation for 1000000 paths...
Numba simulation finished.
Calculating final price/stderr (no VR)...

Numba Plain MC Result:
Price: 0.06437730 ± 0.00005608
Time:  1.01 seconds
Simulations per second: 992917
------------------------------
Starting Numba simulation for 1000000 paths...
Numba simulation finished.
Calculating final price/stderr with Control Variate...
CV calculation finished.

Numba Exact CV Result:
Price: 0.06440607 ± 0.00002985 (g=-0.3937)
Time:  21.76 seconds
Simulations per second: 45959
------------------------------

Variance Reduction Factor (Exact CV vs Plain): 3.53x

Achieved simulations per hour (Exact CV): 165,453,045
Target simulations per hour:             1,000,000
>>> Target performance MET or EXCEEDED! <<<

Estimated speedup vs original (50 sims/sec): 919.2x
>>> Target speedup (5x) ACHIEVED! <<<


In [23]:
print(f"plain: {p0:.6f} ± {e0:.6f}")
print(f"FD-CV:  {p_fd:.6f} ± {e_fd:.6f} (g={g_fd:.4f})")
print(f"Exact:  {p_ex:.6f} ± {e_ex:.6f} (g={g_ex:.4f})")

plain: 0.062768 ± 0.001029
FD-CV:  0.062768 ± 0.000965 (g=-0.0479)
Exact:  0.062768 ± 0.000965 (g=-0.0479)
